# Testing the __Restart & Run All__-ability of Notebooks

We should consider two conditions for notebooks:

1. Those that have formal tests.        
2. Those that don't.

 If no errors are raised during test collection then we know the document will __restart and run all__. `pytest` returns a `SystemError` code of `5` if no tests are found. _It is jarring even though there was no actual failure._

This notebook will use a `pytest` plugin discussed in [Call "--collect-only" programmatically](https://github.com/pytest-dev/pytest/issues/2039).  Our plugin will not run `pytest` if no tests are found.

## Usage 

This notebook exports a CLI to "__smart test__" notebooks.

    ipython -m deathbeds.2018-07-16-Testing-restart-run-all 2018-07-16-Testing-restart-run-all.ipynb
    
[`2018-07-15-Pytest-watchmedo`](2018-07-15-Pytest-watchmedo)  creates a file watcher from this post.    

In [1]:
    import pytest, sys
    from dataclasses import dataclass, field

## A plugin class for pytest 

... to store found tests on an object.  This is based of an issue on [`pytest-dev`](https://github.com/pytest-dev/pytest/issues/2039)

In [2]:
    @dataclass(unsafe_hash=True)
    class TestCollection:
        tests: tuple = field(default_factory=tuple)
        def pytest_collection_modifyitems(self, items):
            for item in items:  self.tests += item.nodeid,

`collect_tests` will allow us to measure the numbers of tests that are found.

In [3]:
    def collect_tests(filename):
        object = TestCollection()
        pytest.main(['--collect-only', filename], plugins=[object])
        return object.tests

In [5]:
    def test_collection(): 
        __file__ = globals().get('__file__', "2018-07-16-Smart-testing-pytest-nbconvert.ipynb")
        collect_tests(__file__)

`smart_test` will make will not run `pytest` if no tests were collected.

    >>> with __import__('IPython').utils.capture.capture_output():
    ...     assert not smart_test('2018-07-16-Testing-restart-run-all.ipynb')

In [6]:
    def smart_test(filename):
        collect_tests(filename) and pytest.main([filename])

## The `__main__` context

The __main__ context is used as a CLI or doctest depending on the caller.

In [7]:
    def _restart_and_run_all_ability():
        if sys.argv[0].endswith(__file__): smart_test(sys.argv[1])
        else:
            from deathbeds import _018_07_16_Testing_restart_run_all
            print(__import__('doctest').testmod(_018_07_16_Testing_restart_run_all))

TestResults(failed=0, attempted=1)
